### Final Project SBA Loan Data
### DSC 630
### Sam Loyd
### November 2020
### Pycaret_Reg

### This code was used to find the best regression model for job creation.

In [1]:
# Required libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as pt
import sklearn
from scipy.stats import pearsonr
import matplotlib.pyplot as matplotlib
from matplotlib.lines import Line2D
from matplotlib import pyplot
pd.options.display.float_format = '{:.2f}'.format
import seaborn 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from IPython.display import Image
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from pycaret.regression import *
# from pycaret.classification import *

In [2]:
raw_sba_df = pd.read_csv("C:\\Users\\Loyd\\Documents\\630Project\\sbapython.csv")
raw_sba_df

,ApprovalFY,Term,NoEmp,CreateJob,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
0,35,84,4,0,60000,48000,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,35,60,2,0,40000,32000,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,35,180,7,0,287000,215250,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
3,35,60,2,0,35000,28000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,35,240,14,7,229000,229000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897105,35,60,6,0,70000,56000,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897106,35,60,6,0,85000,42500,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897107,35,108,26,0,300000,225000,0,1,0,0,...,0,0,0,0,1,1,1,1,0,1
897108,35,60,6,0,75000,60000,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1


#### Normalize the data in case some models require it.  Non-gaussian so normalizing instead of standardizing.

In [3]:
# Normalize the data since its non-Gaussian
# scaler = MinMaxScaler() 
# sba_df[['Term','NoEmp','']] = scaler.fit_transform(data)
sba_df = raw_sba_df.copy()
# data['logarithm_base2'] = np.log2(data['Salary']) 
sba_df[['Term','NoEmp','GrAppv','SBA_Appv']] = np.log1p(sba_df[['Term','NoEmp','GrAppv','SBA_Appv']])
sba_df[['Term','NoEmp','GrAppv','SBA_Appv']] = minmax_scale(sba_df[['Term','NoEmp','GrAppv','SBA_Appv']])

sba_df

,ApprovalFY,Term,NoEmp,CreateJob,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
0,35,0.70,0.17,0,0.48,0.51,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,35,0.65,0.12,0,0.43,0.47,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,35,0.82,0.23,0,0.66,0.68,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
3,35,0.65,0.12,0,0.42,0.45,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,35,0.86,0.29,7,0.64,0.69,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897105,35,0.65,0.21,0,0.50,0.53,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897106,35,0.65,0.21,0,0.52,0.50,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897107,35,0.74,0.36,0,0.67,0.68,0,1,0,0,...,0,0,0,0,1,1,1,1,0,1
897108,35,0.65,0.21,0,0.51,0.54,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1


#### Split the data into 60,20,20 Train, Test Validate Sets.

In [4]:
# Create X from the features
X = sba_df.drop(columns=['MIS_logical','CreateJob'])

# Create y from output
y = sba_df[['MIS_logical','CreateJob']]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [5]:
y_train

,MIS_logical,CreateJob
623129,1,0
589937,1,4
576584,1,0
618776,1,0
466287,1,0
...,...,...
111316,1,0
883883,1,14
374049,1,0
54901,0,0


In [6]:
X_train

,ApprovalFY,Term,NoEmp,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,UrbanRural_0,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
623129,32,0.65,0.23,0.51,0.55,0,1,0,0,1,...,0,0,0,0,0,0,1,1,0,0
589937,41,0.86,0.12,0.61,0.63,0,1,0,0,0,...,0,0,1,1,0,1,1,1,1,0
576584,40,0.89,0.19,0.81,0.82,0,1,0,0,0,...,0,0,1,0,1,0,0,1,1,0
618776,41,0.70,0.27,0.65,0.62,0,1,0,0,0,...,0,0,0,1,1,0,1,0,0,0
466287,29,0.76,0.35,0.65,0.68,0,1,0,0,1,...,1,0,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111316,36,0.76,0.19,0.78,0.79,0,0,1,0,1,...,0,1,1,1,1,1,0,0,0,1
883883,44,0.86,0.08,0.69,0.73,0,0,1,0,0,...,0,1,1,1,0,0,0,1,0,0
374049,47,0.69,0.17,0.42,0.48,0,1,0,0,0,...,0,0,1,1,0,1,1,1,1,0
54901,43,0.57,0.33,0.54,0.52,0,1,0,0,0,...,0,0,0,0,0,0,1,1,1,0


In [7]:
#Pycaret requires them to be in the same dataframe and just labelled
X_train_caret = X_train.copy()
X_train_caret['CreateJob'] = y_train['CreateJob']
X_train_caret

,ApprovalFY,Term,NoEmp,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,UrbanRural_0,...,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13,CreateJob
623129,32,0.65,0.23,0.51,0.55,0,1,0,0,1,...,0,0,0,0,0,1,1,0,0,0
589937,41,0.86,0.12,0.61,0.63,0,1,0,0,0,...,0,1,1,0,1,1,1,1,0,4
576584,40,0.89,0.19,0.81,0.82,0,1,0,0,0,...,0,1,0,1,0,0,1,1,0,0
618776,41,0.70,0.27,0.65,0.62,0,1,0,0,0,...,0,0,1,1,0,1,0,0,0,0
466287,29,0.76,0.35,0.65,0.68,0,1,0,0,1,...,0,1,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111316,36,0.76,0.19,0.78,0.79,0,0,1,0,1,...,1,1,1,1,1,0,0,0,1,0
883883,44,0.86,0.08,0.69,0.73,0,0,1,0,0,...,1,1,1,0,0,0,1,0,0,14
374049,47,0.69,0.17,0.42,0.48,0,1,0,0,0,...,0,1,1,0,1,1,1,1,0,0
54901,43,0.57,0.33,0.54,0.52,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0


In [8]:
# Setup Target
exp_clf = setup(X_train_caret, target='CreateJob')

 
Setup Succesfully Completed.


,Description,Value
0,session_id,4094
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(538266, 69)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,63
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [9]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model.Lasso,True
ridge,Ridge Regression,sklearn.linear_model.Ridge,True
en,Elastic Net,sklearn.linear_model.ElasticNet,True
lar,Least Angle Regression,sklearn.linear_model.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model.OMP,True
br,Bayesian Ridge,sklearn.linear_model.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model.ARDRegression,False


In [10]:
compare_models(exclude=['ransac','svm'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Light Gradient Boosting Machine,2.5110,805.0595,24.0522,0.9840,0.8174,0.6456,2.6934
1,Random Forest,2.3437,907.0703,26.9691,0.9818,0.7628,0.8195,368.5672
2,CatBoost Regressor,2.5369,935.0904,27.3030,0.9814,0.8232,0.7145,34.7115
3,Extreme Gradient Boosting,2.5038,1016.1820,27.9235,0.9794,0.8106,0.7157,43.0497
4,Gradient Boosting Regressor,3.9209,1261.7134,33.5795,0.9750,0.9916,0.5829,133.1734
5,Decision Tree,2.7800,1465.9661,35.1829,0.9701,0.8811,1.2803,11.2244
6,Extra Trees Regressor,3.9291,3538.0158,57.0482,0.9304,0.8383,0.8744,432.3035
7,AdaBoost Regressor,17.6266,13920.7825,94.4928,0.6942,1.5104,1.8994,49.2989
8,K Neighbors Regressor,9.1642,29148.4861,170.2002,0.4267,0.9043,1.2477,47.4085
9,Ridge Regression,25.2379,50559.3897,224.1580,0.0135,2.4966,7.1524,0.3961


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=4094, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)